## This notebook is for data preprocessing and feature generating

In [1]:
import numpy as np # linear algebra
import pandas as pd 
# Any results you write to the current directory are saved as output.
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
from tqdm import tqdm_notebook
from scipy.stats import kurtosis
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

plt.style.use('seaborn')
sns.set(font_scale=1)

In [2]:
random_state = 42
np.random.seed(random_state)
df_train = pd.read_pickle('./train.pkl')
df_test = pd.read_pickle('./test.pkl')

In [3]:
feature = [col for col in df_train.columns if col not in ['ID_code', 'target']]
# feature = [col for col in feature if col not in noise_cols]
train_X = df_train[feature]#.values
test_X = df_test[feature]#.values
y = df_train['target']#.values

In [14]:
public = np.load('public_LB.npy')
private = np.load('private_LB.npy')
real_idx = np.hstack([public[:],private])

In [17]:
fake_idx = np.array(list(set(range(200000))-set(real_idx)))

In [18]:
real_idx = np.hstack([public,private])
real_test = test_X.iloc[real_idx,:]
fake_test = test_X.iloc[fake_idx,:]

In [19]:
merge = df_train[feature].append(real_test[feature])

In [19]:
real_test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
131076,9.2554,-1.0297,11.1124,6.4313,11.3314,-12.7266,6.3105,16.3000,-2.3239,8.1701,...,-0.4578,4.1876,1.1284,0.3507,21.5428,-2.0701,8.0412,8.3893,16.8802,-2.9252
131078,8.1264,-6.3244,12.0427,10.5956,13.0515,2.4566,3.5076,15.2741,0.2167,7.0154,...,14.9504,6.2563,2.0848,0.1502,12.6401,-1.1512,3.2221,9.6693,20.4775,-5.2140
7,17.3035,-2.4212,13.3989,8.3998,11.0777,9.6449,5.9596,17.8477,-4.8068,7.4643,...,4.4676,4.4214,0.9303,1.4994,15.2648,-1.7931,6.5316,10.4855,23.4631,0.7283
131080,11.8092,3.5175,7.2401,7.0770,11.4143,-14.7267,5.6070,12.5889,-0.4833,6.0718,...,-3.9433,10.3476,0.3822,3.6427,15.0925,-0.2888,1.0308,8.2361,20.6083,-7.6494
131081,8.5003,1.9545,7.6174,5.8388,11.2791,-9.4543,5.4782,17.9878,-1.9208,7.1584,...,5.8599,5.1527,1.4673,-4.5566,18.7032,-0.6078,-3.9766,9.4843,15.7983,5.2175


Tried including kurtosis and Isolation Forest to specify anomaly point, but didn't work

In [52]:
# kurtosis_list = []
# for i in range(200):
#     kurtosis_list.append(kurtosis(list(set(real_test.iloc[:,i]))))

In [11]:
# from sklearn.ensemble import IsolationForest
# clf = IsolationForest(behaviour='new', max_samples=100, n_jobs=-1,
#                       random_state=random_state, contamination='auto')

In [49]:
# %%time
# clf.fit(real_test)
# # iso = (train_X)

CPU times: user 128 ms, sys: 220 ms, total: 348 ms
Wall time: 3.39 s


IsolationForest(behaviour='new', bootstrap=False, contamination='auto',
        max_features=1.0, max_samples=100, n_estimators=100, n_jobs=-1,
        random_state=42, verbose=0)

In [9]:
merge.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [20]:
def add_count(df):
    for f in tqdm_notebook(feature):
        count = df[f].value_counts().keys()[(df[f].value_counts()>1)]
        df[f+'_i'] = df[f].apply(lambda x: x if x in count else np.nan)
    return df

In [19]:
# def add_pct(df):
#     for f in tqdm_notebook(feature):
#         count = df[f].value_counts().keys()[(df[f].value_counts()>1)]
#         df[f+'_i'] = df[f].apply(lambda x: round(stats.percentileofscore(df[f],x),3) if x in count else 'nan')
#     return df

# def add_pct2(df):
#     for f in tqdm_notebook(feature):
#         count = df[f].value_counts().keys()[(df[f].value_counts()>1)]
#         df[f+'_i'] = df[f].apply(lambda x: stats.percentileofscore(df[f],x) if x in count else 'nan')
#     return df

## Features 200:400, mark if the data is noised.

In [ ]:
merge_final = add_count(merge_pct)

In [22]:
final_train = merge_final.iloc[:200000,:]
final_test = merge_final.iloc[200000:,:]

In [18]:
np.save('train_0408_noise_raw', final_train.values)
np.save('test_0408_noise_raw', final_test.values)

In [20]:
merge_pct = add_pct975(merge)

## Features 400:600, mark if the data is in the 2.5-97.5 percentile.

In [11]:
low = np.percentile(real_test[f], 0.05)
high = np.percentile(real_test[f], 0.95)

In [13]:
def add_pct975(df):
    for f in tqdm_notebook(feature):
        low = np.percentile(real_test[f], 0.025)
        high = np.percentile(real_test[f], 0.975)
        df[f+'_pct95'] = df[f].apply(lambda x: ((x>low)&(x<high)))
#         df[f+'_pct95'] = df[f].apply(lambda x: round(x,3) if ((x>low)&(x<high)) else 'nan')
    return df

In [ ]:
merge_final = add_pct975(merge_pct)

In [ ]:
final_train = merge_final.iloc[:200000,:]
final_test = merge_final.iloc[200000:,:]

In [23]:
np.save('train_0407_rdpctall_pct975', final_train.values)
np.save('test_0407_rdpctall_pct975', final_test.values)

In [14]:
merge_final.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190_i,var_191_i,var_192_i,var_193_i,var_194_i,var_195_i,var_196_i,var_197_i,var_198_i,var_199_i
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,NaN,NaN,NaN,-1.5121,3.9267,9.5031,17.9974,-8.8104


## Gerating statistics features across different variables.

Since different variables have different scales, before got the features across the features we need to standscale them first.
Then Added variance and 

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

merge_scale = merge.copy()

In [44]:
scaler.fit(real_test[feature])
merge_scale[feature] = scaler.transform(merge[feature])

In [51]:
var_mean = merge_scale.values.mean(axis=1)

var_skew = stats.skew(merge_scale.values,axis=1)
var_sta = np.hstack([np.expand_dims(var_mean,1),np.expand_dims(var_skew,1)])

In [52]:
var_sta.shape

(300000, 2)

In [53]:
var_sta_train = var_sta[:200000]
var_sta_fake = var_sta[200000:]

np.save('var_sta_train', var_sta_train)
np.save('var_sta_fake', var_sta_fake)

In [23]:
final_train = merge_final.iloc[:200000,200:]
fake_test = merge_final.iloc[200000:,200:]

In [73]:
final_train = merge_pct.iloc[:200000,200:]
final_test = merge_pct.iloc[200000:,200:]

In [ ]:
var_mean = merge_scale.values.mean(axis=1)
var_mean = np.apply_along_axis(lambda x: x if ((x>-0.05)&(x<0.05)) else np.nan,1,np.expand_dims(var_mean,1))
var_skew = stats.skew(merge_scale.values,axis=1)
var_skew = np.apply_along_axis(lambda x: x if ((x>-0.1)&(x<0.1)) else np.nan,1,np.expand_dims(var_skew,1))
var_sta = np.hstack([var_mean,var_skew])

In [61]:
final_train = merge.iloc[:200000,:]
final_test = merge.iloc[200000:,:]

In [63]:
np.save('train_X', final_train.values)
np.save('test_X', final_test.values)